In [1]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns
from PIL import Image

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

import pickle

import warnings
warnings.filterwarnings('ignore')

import tidalUtl.PrpUtl as prp
import tidalUtl.EdaUtl as eda

In [2]:
print(torch.__version__) 

1.7.0+cu110


# Version

__ver1__<br>
baseline：CV:0.01465 LB:0.01874<br>
__ver2__<br>
Hyperopt, 2Layer：CV:0.01460 LB:0.01869<br>
__ver3__<br>
3Layer：CV:0.01464 LB:0.01868<br>
__ver4__<br>
MLSMOTE baseline：CV:0.01476 LB:0.01978<br>
__ver5__<br>
2Layer,refactoring：CV:0.01476 LB:0.01869<br>
__ver6__<br>
rankGauss：CV:0.01456 LB:0.01865<br>
__ver7__<br>
labelSmoothing：CV:0.01502 LB:0.01859<br>

# Config

In [3]:
INPUT = "/home/tidal/ML_Data/VinBigData/archive_512_512"
OUTPUT = "/home/tidal/ML_Data/VinBigData/output"

SUBMIT = OUTPUT + "/submittion/"
SAVEMODEL = OUTPUT + "/model/FasterRCNN/"
SAVEOOF = OUTPUT + "/OOF/FasterRCNN/"

SIZE = 512

In [4]:
trainMetaFeature = pd.read_csv(INPUT + '/train_meta.csv')#https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/discussion/207955
sampleSubmission = pd.read_csv(INPUT + '/sample_submission.csv')

trainFeature_ = pd.read_csv(INPUT + '/train.csv')
trainFeature = pd.merge(trainFeature_, trainMetaFeature, on="image_id", how="inner")
del trainFeature_
del trainMetaFeature

In [5]:
trainFeature

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,dim0,dim1
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN,2580,2332
1,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R15,NaN,NaN,NaN,NaN,2580,2332
2,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R16,NaN,NaN,NaN,NaN,2580,2332
3,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN,3159,2954
4,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R13,NaN,NaN,NaN,NaN,3159,2954
...,...,...,...,...,...,...,...,...,...,...
67909,ae86eabab95525b41b8e79883ff1cef9,No finding,14,R5,NaN,NaN,NaN,NaN,3055,2864
67910,ae86eabab95525b41b8e79883ff1cef9,No finding,14,R7,NaN,NaN,NaN,NaN,3055,2864
67911,839b2311ffad9bdd9aa98c74aad6e251,No finding,14,R15,NaN,NaN,NaN,NaN,3000,2801
67912,839b2311ffad9bdd9aa98c74aad6e251,No finding,14,R17,NaN,NaN,NaN,NaN,3000,2801


In [6]:
sampleSubmission

,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1
1,004f33259ee4aef671c2b95d54e4be68,14 1 0 0 1 1
2,008bdde2af2462e86fd373a445d0f4cd,14 1 0 0 1 1
3,009bc039326338823ca3aa84381f17f1,14 1 0 0 1 1
4,00a2145de1886cb9eb88869c85d74080,14 1 0 0 1 1
...,...,...
2995,ff91fb82429a27521bbec8569b041f02,14 1 0 0 1 1
2996,ff9fcc4087ed5e941209aa3fa948e364,14 1 0 0 1 1
2997,ffaa288c8abca300974f043b57d81521,14 1 0 0 1 1
2998,ffc441e0c8b7153844047483a577e7c3,14 1 0 0 1 1


In [7]:
#Seed固定
def seed_everything(seed=42):
    #data取得についてのランダム性固定
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    #cudnnによる演算の安定化(評価値の安定)
    torch.backends.cudnn.deterministic = True
    
    #os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(seed=42)

In [8]:
#HyperParameter
param_space = {'hidden_size1': 512, 
               'hidden_size2': 512, 
               'dropOutRate1': 0.20393004966355735, 
               'dropOutRate2': 0.39170486751620137,
               'leakyReluSlope': 0.01973893854348531,
              }

# Preprocessing

## Func: In & Out Type is DataFrame

## Preprocessing Summary

In [9]:
def preprocessing(param, train):
    trainFeature = train.copy()
    
    #nullを(0,0,1,1)で埋める（classが"No Finding"の場合はNaNになっている）
    trainFeature.fillna(0, inplace=True)
    trainFeature.loc[trainFeature["class_id"] == 14, ["x_max", "y_max"]] = 1.0
    
    #FasterRCNNは背景（正常状態）を0として扱うので"No Finding"を14->0に移行しておく。
    trainFeature["class_id"] = trainFeature["class_id"] + 1
    trainFeature.loc[trainFeature["class_id"] == 15, ["class_id"]] = 0
    
    #リサイズ比(rate_x,rate_y)を追加
    #初期値は1(No findingのサイズは変えないようにするため)
    trainFeature["rate_x"] = 1.
    trainFeature["rate_y"] = 1.
    tmp = SIZE/trainFeature.loc[trainFeature.class_id != 0, ["dim1"]] 
    trainFeature.loc[trainFeature.class_id != 0, ["rate_x"]] = tmp.values.squeeze()
    tmp = SIZE/trainFeature.loc[trainFeature.class_id != 0, ["dim0"]] 
    trainFeature.loc[trainFeature.class_id != 0, ["rate_y"]] = tmp.values.squeeze()
    
    
    #BOXの大きさを調整（x_min, y_min, x_max, y_max）
    #trainFeature.loc[trainFeature.class_id != 0, ["x_min", "y_min", "x_max", "y_max"]] = \
    #    trainFeature.loc[trainFeature.class_id != 0, ["x_min", "y_min", "x_max", "y_max"]]*RESIZERATE
    #trainFeature.loc[trainFeature.class_id != 0, ["x_min", "y_min", "x_max", "y_max"]] = \
    #    trainFeature.loc[trainFeature.class_id != 0, ["x_min", "y_min", "x_max", "y_max"]].round()
    
    return trainFeature

## Work

## Visualization

In [22]:
%%time
trainVsl = preprocessing(param_space, trainFeature)

CPU times: user 21.2 ms, sys: 4.19 ms, total: 25.4 ms
Wall time: 24.4 ms


In [23]:
trainVsl.head(5)

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,dim0,dim1,rate_x,rate_y
0,50a418190bc3fb1ef1633bf9678929b3,No finding,0,R11,0.0,0.0,1.0,1.0,2580,2332,1.0,1.0
1,50a418190bc3fb1ef1633bf9678929b3,No finding,0,R15,0.0,0.0,1.0,1.0,2580,2332,1.0,1.0
2,50a418190bc3fb1ef1633bf9678929b3,No finding,0,R16,0.0,0.0,1.0,1.0,2580,2332,1.0,1.0
3,21a10246a5ec7af151081d0cd6d65dc9,No finding,0,R7,0.0,0.0,1.0,1.0,3159,2954,1.0,1.0
4,21a10246a5ec7af151081d0cd6d65dc9,No finding,0,R13,0.0,0.0,1.0,1.0,3159,2954,1.0,1.0


In [24]:
trainVsl.loc[trainVsl.class_id != 0,]

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,dim0,dim1,rate_x,rate_y
6,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,4,R10,691.0,1375.0,1653.0,1831.0,2336,2080,0.246154,0.219178
7,9a5094b2563a1ef3ff50dc5c7ff71345,Pleural effusion,11,R9,1789.0,1729.0,1875.0,1992.0,2336,2080,0.246154,0.219178
8,9a5094b2563a1ef3ff50dc5c7ff71345,Pleural thickening,12,R9,1789.0,1729.0,1875.0,1992.0,2336,2080,0.246154,0.219178
9,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,4,R9,692.0,1375.0,1657.0,1799.0,2336,2080,0.246154,0.219178
10,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,4,R8,689.0,1313.0,1666.0,1763.0,2336,2080,0.246154,0.219178
...,...,...,...,...,...,...,...,...,...,...,...,...
67774,52951d7de2485aba8ed62629eee4d254,Other lesion,10,R9,303.0,1442.0,383.0,1508.0,2880,2304,0.222222,0.177778
67775,52951d7de2485aba8ed62629eee4d254,Cardiomegaly,4,R8,734.0,1571.0,1614.0,1892.0,2880,2304,0.222222,0.177778
67848,1224f07d895107573588225f692e94f9,Aortic enlargement,1,R10,999.0,716.0,1276.0,988.0,2264,2040,0.250980,0.226148
67849,1224f07d895107573588225f692e94f9,Aortic enlargement,1,R8,1046.0,688.0,1272.0,979.0,2264,2040,0.250980,0.226148


In [25]:
trainVsl["x_min_resize"] = trainVsl.x_min*trainVsl.rate_x
trainVsl.x_min_resize = trainVsl.x_min_resize.round()

trainVsl["y_min_resize"] = trainVsl.y_min*trainVsl.rate_y
trainVsl.y_min_resize = trainVsl.y_min_resize.round()

trainVsl["x_max_resize"] = trainVsl.x_max*trainVsl.rate_x
trainVsl.x_max_resize = trainVsl.x_max_resize.round()
trainVsl.loc[trainVsl.class_id == 0,"x_max_resize"] = 1.

trainVsl["y_max_resize"] = trainVsl.y_max*trainVsl.rate_y
trainVsl.y_max_resize = trainVsl.y_max_resize.round()
trainVsl.loc[trainVsl.class_id == 0,"y_max_resize"] = 1.

In [26]:
trainVsl.loc[trainVsl.class_id != 0,]

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,dim0,dim1,rate_x,rate_y,x_min_resize,y_min_resize,x_max_resize,y_max_resize
6,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,4,R10,691.0,1375.0,1653.0,1831.0,2336,2080,0.246154,0.219178,170.0,301.0,407.0,401.0
7,9a5094b2563a1ef3ff50dc5c7ff71345,Pleural effusion,11,R9,1789.0,1729.0,1875.0,1992.0,2336,2080,0.246154,0.219178,440.0,379.0,462.0,437.0
8,9a5094b2563a1ef3ff50dc5c7ff71345,Pleural thickening,12,R9,1789.0,1729.0,1875.0,1992.0,2336,2080,0.246154,0.219178,440.0,379.0,462.0,437.0
9,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,4,R9,692.0,1375.0,1657.0,1799.0,2336,2080,0.246154,0.219178,170.0,301.0,408.0,394.0
10,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,4,R8,689.0,1313.0,1666.0,1763.0,2336,2080,0.246154,0.219178,170.0,288.0,410.0,386.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67774,52951d7de2485aba8ed62629eee4d254,Other lesion,10,R9,303.0,1442.0,383.0,1508.0,2880,2304,0.222222,0.177778,67.0,256.0,85.0,268.0
67775,52951d7de2485aba8ed62629eee4d254,Cardiomegaly,4,R8,734.0,1571.0,1614.0,1892.0,2880,2304,0.222222,0.177778,163.0,279.0,359.0,336.0
67848,1224f07d895107573588225f692e94f9,Aortic enlargement,1,R10,999.0,716.0,1276.0,988.0,2264,2040,0.250980,0.226148,251.0,162.0,320.0,223.0
67849,1224f07d895107573588225f692e94f9,Aortic enlargement,1,R8,1046.0,688.0,1272.0,979.0,2264,2040,0.250980,0.226148,263.0,156.0,319.0,221.0


In [27]:
trainVsl.loc[trainVsl.class_id == 0,]

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,dim0,dim1,rate_x,rate_y,x_min_resize,y_min_resize,x_max_resize,y_max_resize
0,50a418190bc3fb1ef1633bf9678929b3,No finding,0,R11,0.0,0.0,1.0,1.0,2580,2332,1.0,1.0,0.0,0.0,1.0,1.0
1,50a418190bc3fb1ef1633bf9678929b3,No finding,0,R15,0.0,0.0,1.0,1.0,2580,2332,1.0,1.0,0.0,0.0,1.0,1.0
2,50a418190bc3fb1ef1633bf9678929b3,No finding,0,R16,0.0,0.0,1.0,1.0,2580,2332,1.0,1.0,0.0,0.0,1.0,1.0
3,21a10246a5ec7af151081d0cd6d65dc9,No finding,0,R7,0.0,0.0,1.0,1.0,3159,2954,1.0,1.0,0.0,0.0,1.0,1.0
4,21a10246a5ec7af151081d0cd6d65dc9,No finding,0,R13,0.0,0.0,1.0,1.0,3159,2954,1.0,1.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67909,ae86eabab95525b41b8e79883ff1cef9,No finding,0,R5,0.0,0.0,1.0,1.0,3055,2864,1.0,1.0,0.0,0.0,1.0,1.0
67910,ae86eabab95525b41b8e79883ff1cef9,No finding,0,R7,0.0,0.0,1.0,1.0,3055,2864,1.0,1.0,0.0,0.0,1.0,1.0
67911,839b2311ffad9bdd9aa98c74aad6e251,No finding,0,R15,0.0,0.0,1.0,1.0,3000,2801,1.0,1.0,0.0,0.0,1.0,1.0
67912,839b2311ffad9bdd9aa98c74aad6e251,No finding,0,R17,0.0,0.0,1.0,1.0,3000,2801,1.0,1.0,0.0,0.0,1.0,1.0


In [32]:
print(trainVsl.loc[trainVsl.class_id == 0,"x_min_resize"].value_counts())
print(trainVsl.loc[trainVsl.class_id == 0,"y_min_resize"].value_counts())
print(trainVsl.loc[trainVsl.class_id == 0,"x_max_resize"].value_counts())
print(trainVsl.loc[trainVsl.class_id == 0,"y_max_resize"].value_counts())

0.0    31818
Name: x_min_resize, dtype: int64
0.0    31818
Name: y_min_resize, dtype: int64
1.0    31818
Name: x_max_resize, dtype: int64
1.0    31818
Name: y_max_resize, dtype: int64


In [28]:
print("Train: "+ str(trainVsl.shape))

Train: (67914, 16)


# Fitting

## Config about Fitting

In [14]:
#configは辞書化しておく。
def Config_about_Fitting(train, folds):
    confFitting = {}
    
    #Fitするときに"y"として使う列の列名配列
    confFitting["target_cols"] = target.columns.values.tolist()
    #Fitするときに"X"として使う列の列名配列
    #kfold, id等はここで削除。
    feature_cols = [c for c in folds.columns if c not in confFitting["target_cols"]]
    confFitting["feature_cols"] = [c for c in feature_cols if c not in ['kfold','sig_id']]
    #特徴量、ターゲットのサイズ
    confFitting["num_features"]=len(confFitting["feature_cols"])
    confFitting["num_targets"]=len(confFitting["target_cols"])
    
    return confFitting

## Dataset Classes

In [15]:
#Train,Valid用のデータクラス
class TrainDataset:
    def __init__(self, dataframe, transform_aug=None):
        super().__init__()
        
        self.image_ids = dataframe["image_id"].unique()
        #"width", "height", "xmin", "ymin", "xmax", "ymax", "class"
        self.df = dataframe
        self.trans_aug = trans_aug
    
    def __getitem__(self, index):
        transform = transforms.Compose([
            transforms.ToTensor()
        ])

        # 入力画像の読み込み
        image_id = self.image_ids[index]
        image = Image.open(f"{INPUT}/train/{image_id}.png")
        image = transform(image)

        # ターゲットデータの読み込み
        records = self.df[self.df["image_id"] == image_id]
        #リサイズ後のbox値を計算
        records["x_min_resize"] = records.x_min*records.rate_x
        records.x_min_resize = records.x_min_resize.round()
        records["y_min_resize"] = records.y_min*records.rate_y
        records.y_min_resize = records.y_min_resize.round()
        records["x_max_resize"] = records.x_max*records.rate_x
        records.x_max_resize = records.x_max_resize.round()
        records.loc[records.class_id == 0,"x_max_resize"] = 1.
        records["y_max_resize"] = records.y_max*records.rate_y
        records.y_max_resize = records.y_max_resize.round()
        records.loc[records.class_id == 0,"y_max_resize"] = 1.
        
        boxes = torch.tensor(records[["x_min_resize", "y_min_resize", "x_max_resize", "y_max_resize"]].values, dtype=torch.float32)

        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)

        labels = torch.tensor(records["class_id"].values, dtype=torch.int64)

        iscrowd = torch.zeros((records.shape[0], ), dtype=torch.int64) #全部評価する

        target = {}
        target["boxes"] = boxes #位置座標 
        target["labels"]= labels #物体のclass_id
        target["image_ix"] = torch.tensor([index])
        target["area"] = area #boxの面積
        target["iscrowd"] = iscrowd #True（1）なら評価を無視する

        return image, target, image_id

    def __len__(self):
        return self.image_ids.shape[0]
        
        
#Test用のデータクラス
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            #torch.DataLoaderに入れるための形式
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct
    

## Loss, Metric

In [16]:
#loss
#nn.BCEWithLogitsLoss()

#class LabelSmoothingCrossEntropy(nn.Module):
#    def __init__(self):
#        super(LabelSmoothingCrossEntropy, self).__init__()
#    def forward(self, x, target, smoothing=0.001):
#        confidence = 1. - smoothing
#        logprobs = F.log_softmax(x, dim=-1)
#        bcs_loss = nn.BCEWithLogitsLoss()(x, target)
#        smooth_loss = -logprobs.mean(dim=-1)
#        loss = confidence * bcs_loss + smoothing * smooth_loss
#        return loss.mean()

In [17]:
#metric
#nn.BCEWithLogitsLoss()

## Func: Fitting, Evaluation, Predict

In [18]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
#         print(inputs.shape)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds
   
    

## Model architect

In [19]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator


backbone = torchvision.models.mobilenet_v2(pretrained=True).features
backbone.out_channels = 1280
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

#チュートリアルパクるとここでエラー吐く。([0]を['0']にすれば動く)

#デフォ
roi_pooler =torchvision.ops.MultiScaleRoIAlign(
                featmap_names=['0'],#入力featuremapの名前
                output_size=7,
                sampling_ratio=2)


# put the pieces together inside a FasterRCNN model
Model = FasterRCNN(backbone,
                   num_classes=(len(classes)) + 1,###注意
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)


# Run

## HyperParameter

In [20]:
# HyperParameters
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 25
BATCH_SIZE = 512
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = True

## CV folds

In [21]:
def CV_folds(train, target):
    folds = train.copy()
    
    skf = StratifiedKFold(n_splits=NFOLDS)
    
    for f, (t_idx, v_idx) in enumerate(skf.split(X=train, y=trainTarget)):
        folds.loc[v_idx, 'kfold'] = int(f)
    
    folds['kfold'] = folds['kfold'].astype(int)
    
    return folds

In [22]:
%%time
#Preprocessing Data
trainVsl, testVsl, targetVsl = preprocessing(param_space, trainFeature, testFeature, trainTarget)
#CV folds
foldsVsl = CV_folds(trainVsl, targetVsl)

foldsVsl.head(5)

CPU times: user 979 ms, sys: 55.3 ms, total: 1.03 s
Wall time: 1.03 s


,0,1,2,3,4,5,6,7,8,9,...,automobile,bird,cat,deer,dog,frog,horse,ship,truck,kfold
0,59,43,50,68,98,119,139,145,149,149,...,0,0,0,0,0,1,0,0,0,0
1,154,126,105,102,125,155,172,180,142,111,...,0,0,0,0,0,0,0,0,1,0
2,255,253,253,253,253,253,253,253,253,253,...,0,0,0,0,0,0,0,0,1,0
3,28,37,38,42,44,40,40,24,32,43,...,0,0,0,1,0,0,0,0,0,0
4,170,168,177,183,181,177,181,184,189,189,...,1,0,0,0,0,0,0,0,0,0


In [23]:
foldsVsl.kfold.value_counts()

4    10000
3    10000
2    10000
1    10000
0    10000
Name: kfold, dtype: int64

## Single Fold Running

In [24]:
def run_training(confFitting, Tester, fold, seed, param,
                 folds, train, test, target):
    
    seed_everything(seed)
    
    train = folds
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[confFitting["feature_cols"]].values, train_df[confFitting["target_cols"]].values
    x_valid, y_valid =  valid_df[confFitting["feature_cols"]].values, valid_df[confFitting["target_cols"]].values
    
    train_dataset = TrainDataset(x_train, y_train)
    valid_dataset = TrainDataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=confFitting["num_features"],
        num_targets=confFitting["num_targets"],
        param=param
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    ##### 評価関数 ######
    train_loss_fn = nn.BCEWithLogitsLoss()
    valid_loss_fn = nn.BCEWithLogitsLoss()
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), target.shape[1]))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, train_loss_fn, trainloader, DEVICE)
        valid_loss, valid_preds = valid_fn(model, valid_loss_fn, validloader, DEVICE)
        if Tester:
            print("EPOCH: {:03}: | train_loss: {:.3f}: | valid_loss: {:.3f}".format(epoch, train_loss, valid_loss))
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"{SAVEMODEL}SEED{seed}_FOLD{fold}.pth")
        
        elif(EARLY_STOP == True):
            early_step += 1
            if (early_step >= early_stopping_steps):
                if Tester:
                    print('Early stopping. Best Val loss: {:.3f}'.format(best_loss))
                break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test[confFitting["feature_cols"]].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=confFitting["num_features"],
        num_targets=confFitting["num_targets"],
        param=param
    )
    
    model.load_state_dict(torch.load(f"{SAVEMODEL}SEED{seed}_FOLD{fold}.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    
    return oof, predictions


## K-Fold Running

In [25]:
def run_k_fold(Tester, NFOLDS, seed, param,
              folds, train, test, target, confFitting):
    oof = np.zeros((len(train), confFitting["num_targets"]))
    predictions = np.zeros((len(test), confFitting["num_targets"]))
    
    for fold in range(NFOLDS):
        if Tester:
            print('=' * 20, 'Fold', fold, '=' * 20)
        oof_, pred_ = run_training(confFitting, Tester, fold, seed, param,
                                   folds, train, test, target)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

## CV Evaluation

In [26]:
 def CV_Evaluation(confFitting, oof, target):
    score = []
    
    #cross entropy
    y_true_OH = target[confFitting["target_cols"]].values
    y_pred_proba = oof
    
    score_logloss = 0
    for i in range(confFitting["num_targets"]):
        score_ = log_loss(y_true_OH[:, i], y_pred_proba[:, i]) #問題の評価指標によって変わる。
        score_logloss += score_ / target.shape[1]
        
    print("CV cross entropy: ", score_logloss)
    score.append(score_logloss)
    
    
    #accuracy
    score_accuracy = 0
    y_true = trainTarget.values
    y_pred= np.zeros((trainTarget.shape[0],))
    for i in range(trainTarget.shape[0]):
        #pred_proba->predに変形
        y_pred[i] = np.argmax(oof[i])
    
    score_accuracy = accuracy_score(y_true, y_pred)
    
    print("CV accuracy: ", score_accuracy)
    score.append(score_accuracy)
    
    
    #OOF save
    np.save(SAVEOOF + 'oof', y_pred_proba)
    
    return score

## Postprocessing

In [27]:
# 特になし

## Submit

In [28]:
def Submit(confFitting, predictions, test):
    test[confFitting["target_cols"]] = predictions
    sub = sample_submission.drop(columns=confFitting["target_cols"]).merge(test[['sig_id']+confFitting["target_cols"]], on='sig_id', how='left').fillna(0)
    sub.to_csv(f'{SUBMIT}submission.csv', index=False)

    print("sub.shape" + str(sub.shape))
    
    return

# Execute

In [29]:
def Exec(param):
    
    #Tester(True/False)
    Tester = True
    
    #Preprocessing Data
    train, test, target = preprocessing(param, trainFeature, testFeature, trainTarget)
    
    #CV folds
    folds = CV_folds(train, target)
    
    #Config about Fitting
    confFitting = Config_about_Fitting(train, test, target, folds)
    
    # Averaging on multiple SEEDS
    SEED = [42]
    oof = np.zeros((len(train), confFitting["num_targets"]))
    predictions = np.zeros((len(test), confFitting["num_targets"]))
    
    ### RUN ###
    for seed in SEED:
        if Tester:
            print('~' * 20, 'SEED', seed, '~' * 20)
        oof_, predictions_ = run_k_fold(Tester, NFOLDS, seed, param,
                                       folds, train, test, target, confFitting)
        oof += oof_ / len(SEED)
        predictions += predictions_ / len(SEED)
    
    #CV 評価
    score = CV_Evaluation(confFitting, oof, target)
    
    # 課題提出
    #Submit(confFitting, predictions, test)
    
    return score, oof, predictions


In [30]:
%%time
score, oof, predictions = Exec(param_space)

~~~~~~~~~~~~~~~~~~~~ SEED 42 ~~~~~~~~~~~~~~~~~~~~
==================== Fold 0 ====================
EPOCH: 000: | train_loss: 0.646: | valid_loss: 0.344
EPOCH: 001: | train_loss: 0.272: | valid_loss: 0.255
EPOCH: 002: | train_loss: 0.243: | valid_loss: 0.233
EPOCH: 003: | train_loss: 0.232: | valid_loss: 0.224
EPOCH: 004: | train_loss: 0.225: | valid_loss: 0.217
EPOCH: 005: | train_loss: 0.220: | valid_loss: 0.215
EPOCH: 006: | train_loss: 0.217: | valid_loss: 0.214
EPOCH: 007: | train_loss: 0.214: | valid_loss: 0.209
EPOCH: 008: | train_loss: 0.211: | valid_loss: 0.208
EPOCH: 009: | train_loss: 0.208: | valid_loss: 0.209
EPOCH: 010: | train_loss: 0.205: | valid_loss: 0.207
EPOCH: 011: | train_loss: 0.204: | valid_loss: 0.205
EPOCH: 012: | train_loss: 0.200: | valid_loss: 0.201
EPOCH: 013: | train_loss: 0.197: | valid_loss: 0.199
EPOCH: 014: | train_loss: 0.194: | valid_loss: 0.198
EPOCH: 015: | train_loss: 0.190: | valid_loss: 0.196
EPOCH: 016: | train_loss: 0.188: | valid_loss: 0.194


In [31]:
oof[3]

array([0.0167854 , 0.00720065, 0.05208615, 0.02029384, 0.2257587 ,
       0.0255843 , 0.61460459, 0.00513765, 0.0029205 , 0.00281287])

In [32]:
oof[3][np.argmax(oof[3])]

0.6146045923233032

In [33]:
np.argmax(oof[2])

9

In [34]:
oof.shape

(50000, 10)

In [35]:
_, _, targetVsl = preprocessing(param_space, trainFeature, testFeature, trainTarget)

In [36]:
trainTarget.values

array([6, 9, 9, ..., 9, 1, 1])

In [37]:
y_pred= np.zeros((trainTarget.shape[0],))
for i in range(trainTarget.shape[0]):
    y_pred[i] = np.argmax(oof[i])

In [38]:
y_pred

array([6., 9., 9., ..., 9., 1., 1.])

# Predict

In [39]:
def run_predict(confFitting, param, test, target, fold, seed):
    
    seed_everything(seed)
  
    #--------------------- PREDICTION---------------------
    x_test = test[confFitting["feature_cols"]].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=confFitting["num_features"],
        num_targets=confFitting["num_targets"],
        param=param
    )
    
    model.load_state_dict(torch.load(f"{SAVEMODEL}SEED{seed}_FOLD{fold}.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    
    return predictions


In [40]:
def run_k_fold_predict(confFitting, test, target, param, Tester, NFOLDS, seed):
    predictions = np.zeros((len(test), confFitting["num_targets"]))
    
    for fold in range(NFOLDS):
        if Tester:
            print('=' * 20, 'Fold', fold, '=' * 20)
        pred_ = run_predict(confFitting, param, test, target, fold, seed)
        
        predictions += pred_ / NFOLDS
        
    return predictions

In [41]:
def SubmitPredict(confFitting, predictions, test, prefix):
    test[confFitting["target_cols"]] = predictions
    sub = sample_submission.drop(columns=confFitting["target_cols"]).merge(test[['sig_id']+confFitting["target_cols"]], on='sig_id', how='left').fillna(0)
    sub.to_csv(f'{SUBMIT}{prefix}submission.csv', index=False)

    print("sub.shape" + str(sub.shape))
    
    return

In [42]:
def Predict(param):
    #Tester(True/False)
    Tester = False
    
    #Preprocessing Data
    train, test, target = preprocessing(param, trainFeature, testFeature, trainTargetScored)
    
    #CV folds
    folds = CV_folds(train, target)
    
    #Config about Fitting
    confFitting = Config_about_Fitting(train, test, target, folds)
    
    # Averaging on multiple SEEDS
    SEED = [0, 1, 2, 3 ,4, 5]
    predictions = np.zeros((len(test), confFitting["num_targets"]))
    
    ### RUN ###
    for seed in SEED:
        if Tester:
            print('~' * 20, 'SEED', seed, '~' * 20)
        predictions_ = run_k_fold_predict(confFitting, test, target, param, Tester, NFOLDS, seed)
        predictions += predictions_ / len(SEED)
    
    # 課題提出
    prefix = "Pytorch"
    SubmitPredict(confFitting, predictions, test, prefix)
    
    return

In [43]:
%%time
#Predict(param_space)

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs


# Hyperparameter Tuning

In [44]:
#hyperopt
from hyperopt import fmin, tpe, hp, rand, Trials

In [45]:
def HOptExec(param):
    #Tester(True/False)
    Tester = False
    
    #Preprocessing Data
    train, test, target = preprocessing(param, trainFeature, testFeature, trainTargetScored)
    
    #CV folds
    folds = CV_folds(train, target)
    
    #Config about Fitting
    confFitting = Config_about_Fitting(train, test, target, folds)
    
    # Averaging on multiple SEEDS
    SEED = [0, 1, 2, 3 ,4, 5]
    oof = np.zeros((len(train), confFitting["num_targets"]))
    predictions = np.zeros((len(test), confFitting["num_targets"]))
    
    ### RUN ###
    for seed in SEED:
        if Tester:
            print('~' * 20, 'SEED', seed, '~' * 20)
        oof_, predictions_ = run_k_fold(Tester, NFOLDS, seed, param,
                                       folds, train, test, target, confFitting)
        oof += oof_ / len(SEED)
        predictions += predictions_ / len(SEED)
    
    #CV 評価
    score = CV_Evaluation(confFitting, oof, train, target)
    
    # 課題提出
    #Submit(confFitting, predictions, test)
    
    return score

In [46]:
%%time

#param_space = {'hidden_size1': 512, 
#               'hidden_size2': 512, 
#               'dropOutRate1': 0.20393004966355735, 
#               'dropOutRate2': 0.39170486751620137,
#               'rankGauss_n_quantiles': 488.0393350201078,
#               'leakyReluSlope': hp.uniform('leakyReluSlope', 1e-3, 1e-1),
#              }
#
#trials = Trials()
#
#hopt = fmin(fn = HOptExec, 
#            space = param_space, 
#            algo = tpe.suggest, 
#            max_evals = 15, 
#            #timeout = 8.9 * 60 * 60, 
#            trials = trials, 
#           )
#
#print(hopt)

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.34 µs
